# Open Alex Data Loader Notebook

In [1]:
#initialize data set
from pyspark.sql import SparkSession
import pyspark.pandas as ps
import pyspark.sql.functions as F
from pyspark.sql.functions import udf

spark = (SparkSession
     .builder
     .master('local[*]')
     .config('spark.sql.execution.arrow.pyspark.enabled', 'true')
     .getOrCreate())
spark.sparkContext.setLogLevel('OFF')

/usr/local/spark/python/pyspark/pandas/__init__.py:49: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/14 23:56:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/14 23:56:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Data Loading and Conversion to Parquet

### Sampled 20 gb dataset

In [ ]:
# #20 gb dataset 
# root = "/mnt/data/public/openalex/data/authors/"
# paths = [root + 'updated_date=2022-0[4-8]-0[1-9]/*', 
#          root + 'updated_date=2022-0[4-8]-1[0-9]/*', 
#          root + 'updated_date=2022-0[4-8]-2[0-2]/*']
# df20 = spark.read.json(paths) #
# df20.write.parquet('projectfile_recued.parquet')
#df20_read = spark.read.parquet('projectfile_recued.parquet')

### Full Data

In [ ]:
# #run this once for full data set
# file = '/mnt/data/public/openalex/data/authors/*/*'

# df = spark.read.json(file)
# df.write.parquet('projectfile.parquet')

### 50 MB data set used for query writing

In [ ]:
#run this once for small data
# file = '/mnt/data/public/openalex/data/authors/updated_date=2022-07-24/*'

# df = spark.read.json(file)
# df.write.parquet('project_small.parquet')

### Load parquet files

Ensure that the extraction and conversion is done before running

In [9]:
#df2 = spark.read.parquet('project_small.parquet')
#df2 = spark.read.parquet('projectfile_recued.parquet')
df2 = spark.read.parquet('projectfile.parquet')

dir_ = 'processed_full/'
#dir_ = ''

In [3]:
df2.printSchema()

root
 |-- cited_by_count: long (nullable = true)
 |-- counts_by_year: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- cited_by_count: long (nullable = true)
 |    |    |-- works_count: long (nullable = true)
 |    |    |-- year: long (nullable = true)
 |-- created_date: string (nullable = true)
 |-- display_name: string (nullable = true)
 |-- display_name_alternatives: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- ids: struct (nullable = true)
 |    |-- mag: long (nullable = true)
 |    |-- openalex: string (nullable = true)
 |    |-- orcid: string (nullable = true)
 |    |-- scopus: string (nullable = true)
 |    |-- twitter: string (nullable = true)
 |    |-- wikipedia: string (nullable = true)
 |-- last_known_institution: struct (nullable = true)
 |    |-- country_code: string (nullable = true)
 |    |-- display_name: string (nullable = true)
 |    |-- id: string (nullable = true)

In [4]:
df2.count()

253779827

In [11]:
df_samples = df2.limit(10).toPandas()
df_samples.to_csv(dir_ + 'samples.csv', header=True, index=False)
df_samples

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:114: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: ArrayType(StructType([StructField('cited_by_count', LongType(), True), StructField('works_count', LongType(), True), StructField('year', LongType(), True)]), True)
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


,cited_by_count,counts_by_year,created_date,display_name,display_name_alternatives,id,ids,last_known_institution,most_cited_work,orcid,updated_date,works_api_url,works_count,x_concepts
0,0,[],2017-06-30,compositeur . Compositeur,[],https://openalex.org/A2633105175,"(2633105175, https://openalex.org/A2633105175,...",None,Girondine : valse lente [pour piano] : op. 37 ...,None,2022-08-31T14:08:53.420550,https://api.openalex.org/works?filter=author.i...,1,"[(Humanities, https://openalex.org/C15708023, ..."
1,0,[],2017-06-30,S Sher,[],https://openalex.org/A2636400655,"(2636400655, https://openalex.org/A2636400655,...",None,Nematodes and bacteria on rose: Root-lesion ne...,None,2022-08-31T14:08:53.425739,https://api.openalex.org/works?filter=author.i...,1,"[(Geometry, https://openalex.org/C2524010, 1, ..."
2,0,"[(0, 1, 2016)]",2017-06-30,Владислав Яценко,[],https://openalex.org/A2636811276,"(2636811276, https://openalex.org/A2636811276,...",None,"История Украины by Игорь Данилевский, Татьяна ...",None,2022-08-31T14:08:53.427935,https://api.openalex.org/works?filter=author.i...,1,"[(Computer science, https://openalex.org/C4100..."
3,0,[],2017-06-30,Mícheál Briody,[],https://openalex.org/A2637226189,"(2637226189, https://openalex.org/A2637226189,...","(GB, Language Technology Centre, https://opena...",La mission de Séamus Ó Duilearga : sauvegarder...,None,2022-08-31T14:08:53.432468,https://api.openalex.org/works?filter=author.i...,1,"[(Ethnology, https://openalex.org/C2549261, 1,..."
4,2,"[(1, 0, 2017), (1, 0, 2012)]",2017-06-30,Francisco Caudet Roca,[],https://openalex.org/A2638532457,"(2638532457, https://openalex.org/A2638532457,...",None,El Mono Azul y el romancero de la guerra civil...,None,2022-08-31T14:08:53.435542,https://api.openalex.org/works?filter=author.i...,1,"[(Humanities, https://openalex.org/C15708023, ..."
5,0,[],2017-06-30,本道 佐々木,[],https://openalex.org/A2639591104,"(2639591104, https://openalex.org/A2639591104,...",None,"A-2Barr bodyによる子宮癌の新スクリーニング法(婦人科その(1), 第17回日本臨...",None,2022-08-31T14:08:53.436849,https://api.openalex.org/works?filter=author.i...,1,[]
6,0,[],2017-06-30,隆 重松,[],https://openalex.org/A2640222087,"(2640222087, https://openalex.org/A2640222087,...",None,副甲状腺ホルモン(PTH)測定法の進歩と問題点および今後の課題 : 第47回日本透析医学会シ...,None,2022-08-31T14:08:53.438464,https://api.openalex.org/works?filter=author.i...,3,"[(Computer science, https://openalex.org/C4100..."
7,0,[],2017-06-30,Boucher de Perthes,[],https://openalex.org/A2640373672,"(2640373672, https://openalex.org/A2640373672,...",None,Lettre de Jacques Boucher de Crèvecoeur de Per...,None,2022-08-31T14:08:53.440648,https://api.openalex.org/works?filter=author.i...,1,"[(Humanities, https://openalex.org/C15708023, ..."
8,0,[],2017-06-30,Córdova Fa,[],https://openalex.org/A2640459586,"(2640459586, https://openalex.org/A2640459586,...",None,Rupture of the interventricular septum after a...,None,2022-08-31T14:08:53.444611,https://api.openalex.org/works?filter=author.i...,1,"[(Psychology, https://openalex.org/C15744967, ..."
9,0,[],2017-06-30,박길준,[],https://openalex.org/A2641026821,"(2641026821, https://openalex.org/A2641026821,...",None,상추에서의 싸리수염진딧물(Aulacorthum solani)의 발육과 발육모형 (2...,None,2022-08-31T14:08:53.449805,https://api.openalex.org/works?filter=author.i...,2,"[(Psychology, https://openalex.org/C15744967, ..."


In [10]:
df_desc = df2.describe().toPandas()
df_desc.to_csv(dir_ + 'desc.csv', header=True, index=False)
df_desc

,summary,cited_by_count,created_date,display_name,id,most_cited_work,orcid,updated_date,works_api_url,works_count
0,count,253779827,253779827,253568630,253779827,213480810,4493049,253779827,253779827,253779827
1,mean,31.829215341848272,None,Infinity,None,None,None,None,None,2.3682596528840727
2,stddev,586.4274749064085,None,NaN,None,None,None,None,None,15.90174538780119
3,min,0,2016-06-24,"\t\tL.Hemalatha*, Dr.I.Elizabeth Shanthi",https://openalex.org/A1000000099,\tPERSONAL LOGO FOR A CREATION FIRM OR ATELIER...,https://orcid.org/0---,2022-04-28T14:16:02.478578,https://api.openalex.org/works?filter=author.i...,0
4,max,592203,2022-09-15,􀚏􁲊􀒓 FENG XIAOCAI,https://openalex.org/A999999963,􀭀􀔓􁮷􀓪􂲴􄘭􂉲􃠷􀽒􁜣: (2018),https://orcid.org/9999-9999-9999-9999,2022-09-15T16:48:09.699476,https://api.openalex.org/works?filter=author.i...,94409


In [6]:

# df_summary = df2.summary().toPandas()
# df_summary.to_csv(dir_ + 'summary.csv', header=True, index=False)
# df_summary

,summary,cited_by_count,created_date,display_name,id,orcid,updated_date,works_api_url,works_count
0,count,454949,454949,454107,454949,5989,454949,454949,454949
1,mean,2.6475363172575386,None,None,None,None,None,None,1.791506300706233
2,stddev,9.318778400802668,None,None,None,None,None,None,5.538873780283586
3,min,0,2016-06-24,Alaa-Rashad,https://openalex.org/A1000155635,https://orcid.org/0000-0001-5004-3744,2022-07-24T00:00:00.045906,https://api.openalex.org/works?filter=author.i...,1
4,25%,0,None,None,None,None,None,None,1
5,50%,0,None,None,None,None,None,None,1
6,75%,1,None,None,None,None,None,None,1
7,max,406,2022-05-27,� ������,https://openalex.org/A999732524,https://orcid.org/0000-0003-4997-8862,2022-07-24T23:59:58.719968,https://api.openalex.org/works?filter=author.i...,809


## Spark Dataframe Queries and Conversion to Pandas then CSV Export

The set of code below are the Spark Data Frame Queries and the following cell has the same code where we extended it to `.to_csv()` for the extraction.

In [15]:
# Top works count per author
(df2.select('id','display_name', 'works_count', 'cited_by_count',
            'last_known_institution.display_name', 
            'last_known_institution.country_code')
    .sort(F.desc('works_count'))
    .limit(100).toPandas())

,id,display_name,works_count,display_name,country_code
0,https://openalex.org/A2479313101,Charles Thomas Parker,94409,NamesforLife,US
1,https://openalex.org/A2307396757,George M. Garrity,87714,NamesforLife,US
2,https://openalex.org/A2646173761,Dorothea Taylor,18954,NamesforLife,US
3,https://openalex.org/A1979746700,Panos M. Pardalos,12006,University of Florida,US
4,https://openalex.org/A2777773917,Xin Li,10830,Center for Excellence in Tibetan Plateau Earth...,CN
...,...,...,...,...,...
95,https://openalex.org/A2508859861,Wei Huang,2776,Peking University,CN
96,https://openalex.org/A2443744265,Johann Sebastian Bach,2771,None,None
97,https://openalex.org/A2109091881,Peter G. Jones,2757,University of Cambridge,GB
98,https://openalex.org/A2154542888,Hyunyong Kim,2749,Vanderbilt University,US


In [21]:
# Top works count per author export

(df2.select('id','display_name', 'works_count', 'cited_by_count',
            'last_known_institution.display_name', 
            'last_known_institution.country_code')
    .sort(F.desc('works_count'))
    .limit(100).toPandas()
    .to_csv(dir_ + 'top100_work_count.csv', header=True, index=False))

In [10]:
# Top ciration count per author

(df2.select('id','display_name', 'works_count', 'cited_by_count',
            'last_known_institution.display_name', 
            'last_known_institution.country_code')
   .sort(F.desc('cited_by_count'))
   .limit(100).toPandas())

,id,display_name,works_count,cited_by_count,display_name,country_code
0,https://openalex.org/A2557216553,Eric S. Lander,866,592203,Broad Institute,US
1,https://openalex.org/A1937107987,Douglas G. Altman,1212,527944,Nuffield Orthopaedic Centre,GB
2,https://openalex.org/A2777773917,Xin Li,10830,427095,Center for Excellence in Tibetan Plateau Earth...,CN
3,https://openalex.org/A1793908362,Walter C. Willett,2656,423318,Harvard University,US
4,https://openalex.org/A204361474,David Moher,957,414185,Ottawa Hospital Research Institute,CA
...,...,...,...,...,...,...
95,https://openalex.org/A2608928330,Wei Zhang,8434,190615,Chongqing Institute of Green and Intelligent T...,CN
96,https://openalex.org/A1802456137,Jens K. Nørskov,779,190131,Technical University of Denmark,DK
97,https://openalex.org/A716274308,Nicholas J. Wareham,2075,189797,MRC Epidemiology Unit,GB
98,https://openalex.org/A1900504705,Jost B. Jonas,1469,189403,Heidelberg University,DE


In [22]:
# Top ciration count per author export
(df2.select('id','display_name', 'works_count', 'cited_by_count',
            'last_known_institution.display_name', 
            'last_known_institution.country_code')
   .sort(F.desc('cited_by_count'))
   .limit(100).toPandas()
   .to_csv(dir_ + 'top100_cite_count.csv', header=True, index=False))

In [32]:
# Total work and citations per year

(df2.withColumn('expl_cnt_yr', F.explode('counts_by_year'))
    .select(F.col('expl_cnt_yr').getItem('year').alias('year'), 
                     F.col('expl_cnt_yr').getItem('works_count').alias('work'), 
                     F.col('expl_cnt_yr').getItem('cited_by_count').alias('cite'))
    .groupby('year').agg(F.sum(F.col('work')), F.sum(F.col('cite')))
    .sort(F.desc('sum(work)'))
    .toPandas())



,year,sum(work),sum(cite)
0,2021,89510,87758
1,2020,55017,65529
2,2019,37008,53524
3,2016,36099,57353
4,2015,34180,59686
5,2014,32841,59997
6,2017,32839,55687
7,2013,30969,57383
8,2012,29400,53061
9,2018,28214,47684


In [19]:
# Total work and citations per year export

(df2.withColumn('expl_cnt_yr', F.explode('counts_by_year'))
    .select(F.col('expl_cnt_yr').getItem('year').alias('year'), 
                     F.col('expl_cnt_yr').getItem('works_count').alias('work'), 
                     F.col('expl_cnt_yr').getItem('cited_by_count').alias('cite'))
    .groupby('year').agg(F.sum(F.col('work')), F.sum(F.col('cite')))
    .sort(F.desc('sum(work)'))
    .toPandas()
    .to_csv(dir_ + 'yearly_trend.csv', header=True, index=False))

In [67]:
# Top author based on work count per year

(df2.withColumn('expl_cnt_yr', F.explode('counts_by_year'))
    .select('id', 'display_name',  
                     F.col('last_known_institution.id').alias('inst_id'),
                     F.col('last_known_institution.display_name').alias('inst_display_name'),
                     'last_known_institution.country_code',
                     F.col('expl_cnt_yr').getItem('year').alias('year'), 
                     F.col('expl_cnt_yr').getItem('works_count').alias('work'))
    .groupby('year')
    .agg(F.max(F.col('work')).alias("Max Work"),
         F.first("id").alias("id"),
         F.first("display_name").alias("display_name"),
         F.first("inst_id").alias("inst_id"),
         F.first("inst_display_name").alias("inst_display_name"),
         F.first("country_code").alias("country_code"))
    .toPandas())



,year,Max Work,id,display_name,inst_id,inst_display_name,country_code
0,2012,127,https://openalex.org/A490990781,Reinhard Dipl.-Ing. Großheim,None,None,None
1,2013,139,https://openalex.org/A2772260047,Leonardo Mairene Muniz,None,None,None
2,2014,199,https://openalex.org/A490990781,Reinhard Dipl.-Ing. Großheim,None,None,None
3,2015,260,https://openalex.org/A490990781,Reinhard Dipl.-Ing. Großheim,None,None,None
4,2016,300,https://openalex.org/A490990781,Reinhard Dipl.-Ing. Großheim,None,None,None
5,2017,185,https://openalex.org/A2085018320,Daverson Bordin Canterle,https://openalex.org/I130442723,Federal University of Rio Grande do Sul,BR
6,2018,298,https://openalex.org/A231278954,Sun Yarning,https://openalex.org/I162868743,Tianjin University,CN
7,2019,126,https://openalex.org/A4272990740,Jean-Pierre Serre,None,None,None
8,2020,95,https://openalex.org/A244019625,Baki Karaboece,https://openalex.org/I198068145,Scientific and Technological Research Council ...,TR
9,2021,80,https://openalex.org/A4208781146,Hidekazu SUZUKI,None,None,None


In [20]:
# Top author based on work count per year export

(df2.withColumn('expl_cnt_yr', F.explode('counts_by_year'))
    .select('id', 'display_name',  
                     F.col('last_known_institution.id').alias('inst_id'),
                     F.col('last_known_institution.display_name').alias('inst_display_name'),
                     'last_known_institution.country_code',
                     F.col('expl_cnt_yr').getItem('year').alias('year'), 
                     F.col('expl_cnt_yr').getItem('works_count').alias('work'))
    .groupby('year')
    .agg(F.max(F.col('work')).alias("Max Work"),
         F.first("id").alias("id"),
         F.first("display_name").alias("display_name"),
         F.first("inst_id").alias("inst_id"),
         F.first("inst_display_name").alias("inst_display_name"),
         F.first("country_code").alias("country_code"))
    .toPandas()
    .to_csv(dir_ + 'max_author_year_work.csv', header=True, index=False))



In [70]:
# Top author based on citation count per year

(df2.withColumn('expl_cnt_yr', F.explode('counts_by_year'))
    .select('id', 'display_name',  
                     F.col('last_known_institution.id').alias('inst_id'),
                     F.col('last_known_institution.display_name').alias('inst_display_name'),
                     'last_known_institution.country_code',
                     F.col('expl_cnt_yr').getItem('year').alias('year'), 
                     F.col('expl_cnt_yr').getItem('cited_by_count').alias('cite'))
    .groupby('year')
    .agg(F.max(F.col('cite')).alias("cite"),
         F.first("id").alias("id"),
         F.first("display_name").alias("display_name"),
         F.first("inst_id").alias("inst_id"),
         F.first("inst_display_name").alias("inst_display_name"),
         F.first("country_code").alias("country_code"))
    .toPandas())



,year,cite,id,display_name,inst_id,inst_display_name,country_code
0,2012,47,https://openalex.org/A490990781,Reinhard Dipl.-Ing. Großheim,None,None,None
1,2013,34,https://openalex.org/A2772260047,Leonardo Mairene Muniz,None,None,None
2,2014,98,https://openalex.org/A490990781,Reinhard Dipl.-Ing. Großheim,None,None,None
3,2015,79,https://openalex.org/A490990781,Reinhard Dipl.-Ing. Großheim,None,None,None
4,2016,58,https://openalex.org/A490990781,Reinhard Dipl.-Ing. Großheim,None,None,None
5,2017,25,https://openalex.org/A2085018320,Daverson Bordin Canterle,https://openalex.org/I130442723,Federal University of Rio Grande do Sul,BR
6,2018,31,https://openalex.org/A231278954,Sun Yarning,https://openalex.org/I162868743,Tianjin University,CN
7,2019,122,https://openalex.org/A4272990740,Jean-Pierre Serre,None,None,None
8,2020,49,https://openalex.org/A244019625,Baki Karaboece,https://openalex.org/I198068145,Scientific and Technological Research Council ...,TR
9,2021,75,https://openalex.org/A4208781146,Hidekazu SUZUKI,None,None,None


In [21]:
# Top author based on citation count per year export

(df2.withColumn('expl_cnt_yr', F.explode('counts_by_year'))
    .select('id', 'display_name',  
                     F.col('last_known_institution.id').alias('inst_id'),
                     F.col('last_known_institution.display_name').alias('inst_display_name'),
                     'last_known_institution.country_code',
                     F.col('expl_cnt_yr').getItem('year').alias('year'), 
                     F.col('expl_cnt_yr').getItem('cited_by_count').alias('cite'))
    .groupby('year')
    .agg(F.max(F.col('cite')).alias("cite"),
         F.first("id").alias("id"),
         F.first("display_name").alias("display_name"),
         F.first("inst_id").alias("inst_id"),
         F.first("inst_display_name").alias("inst_display_name"),
         F.first("country_code").alias("country_code"))
    .toPandas()
    .to_csv(dir_ + 'max_author_year_cite.csv', header=True, index=False))



In [22]:
# UDF code to map the id's of the accounts 

def soc_info():
    @udf
    def get_struct(ids, index):
        val = None
        fields = ['mag', 'openalex', 'orcid', 'scopus', 'twitter']
        if (ids.asDict().get(fields[index]) is not None and 
            ids[fields[index]] is not None):
            val = ids.mag

        return val
    return get_struct

@udf
def get_struct(ids, index):

    val = None

    fields = ['mag', 'openalex', 'orcid', 'scopus', 'twitter']

    if (ids.asDict().get(fields[index]) is not None and 
        ids[fields[index]] is not None):
        val = ids.mag

    return val

In [23]:
# Spark Dataframe query that is cached to get count of each social media ID

df_soc= (df2.select(
            soc_info()('ids', F.lit(0)).alias('mag'),
            soc_info()('ids', F.lit(1)).alias('openalex'),
            soc_info()('ids', F.lit(2)).alias('orcid'),
            soc_info()('ids', F.lit(3)).alias('scopus'),
            soc_info()('ids', F.lit(4)).alias('twitter')
           ).cache())

mag_count = (df_soc.select('mag')
                  .where(F.col('mag') != 'None')
                  .count())
openalex_count = (df_soc.select('openalex')
                  .where(F.col('openalex') != 'None')
                  .count())
orcid_count = (df_soc.select('orcid')
                  .where(F.col('orcid') != 'None')
                  .count())
scopus_count = (df_soc.select('scopus')
                  .where(F.col('scopus') != 'None')
                  .count())
twitter_count = (df_soc.select('twitter')
                  .where(F.col('twitter') != 'None')
                  .count())

# Dataframe for the total
data = [("MS Academic Graph", mag_count), 
        ("Open Alex", openalex_count),
        ("ORCID", orcid_count),
        ("Scopus", scopus_count),
        ("Twitter", twitter_count)]
df = spark.createDataFrame(data, ["Social Media", "Count"])
df.toPandas()

,Social Media,Count
0,MS Academic Graph,206216725
1,Open Alex,206216725
2,ORCID,3755377
3,Scopus,690705
4,Twitter,287


In [24]:
# Export social media count 
df.toPandas().to_csv(dir_ + 'soc_med_author.csv', header=True, index=False)

In [25]:
# Spark Dataframe query that is cached to get the insitution details of the authors

df_school = (df2.select('last_known_institution.id',
                        'last_known_institution.display_name',
                        'last_known_institution.country_code',
                        'last_known_institution.ror',
                        'last_known_institution.type',
                        'works_count',
                        'cited_by_count'
                       ).where(F.col('last_known_institution.id') != 'null').cache())

In [89]:
# Utilized cached sparked dataframe for top 100 author count instituion 

df_school.groupby(['id', 'display_name'])
         .count().sort(F.desc('count')).limit(100).toPandas()

,id,display_name,count
0,https://openalex.org/I19820366,Chinese Academy of Sciences,933
1,https://openalex.org/I1313323035,Russian Academy of Sciences,819
2,https://openalex.org/I76130692,Zhejiang University,444
3,https://openalex.org/I183067930,Shanghai Jiao Tong University,425
4,https://openalex.org/I47720641,Huazhong University of Science and Technology,362
...,...,...,...
95,https://openalex.org/I219193219,Purdue University West Lafayette,121
96,https://openalex.org/I107720978,Ain Shams University,120
97,https://openalex.org/I87216513,Michigan State University,120
98,https://openalex.org/I1174212,University of Southern California,120


In [26]:
# Utilized cached sparked dataframe for top 100 author count instituion export

(df_school.groupby(['id', 'display_name'])
          .count().sort(F.desc('count')).limit(100)
          .toPandas()
          .to_csv(dir_ + 'top_institution.csv', header=True, index=False))

In [95]:
# Utilized cached sparked dataframe for top countries author count 

(df_school.where(F.col('country_code') != 'null')
          .groupby(['country_code']).count()
          .sort(F.desc('count')).toPandas())

,country_code,count
0,CN,26415
1,US,23215
2,JP,5515
3,GB,5354
4,IN,5065
...,...,...
169,GN,1
170,ST,1
171,SC,1
172,KY,1


In [27]:
# Utilized cached sparked dataframe for top countries author count export
(df_school.where(F.col('country_code') != 'null')
          .groupby(['country_code']).count()
          .sort(F.desc('count')).toPandas()
          .to_csv(dir_ + 'top_countries.csv', header=True, index=False))

In [98]:
# Utilized cached sparked dataframe for top countries work count 

(df_school.where(F.col('country_code') != 'null')
          .groupby(['country_code']).agg(F.sum('works_count')
          .alias('works_count'))
          .sort(F.desc('works_count')).toPandas())

,country_code,works_count
0,US,44319
1,CN,43354
2,JP,15302
3,BR,10452
4,GB,10169
...,...,...
169,BM,1
170,GN,1
171,SC,1
172,KY,1


In [28]:
# Utilized cached sparked dataframe for top countries work count export

(df_school.where(F.col('country_code') != 'null')
          .groupby(['country_code']).agg(F.sum('works_count')
          .alias('works_count'))
          .sort(F.desc('works_count')).toPandas()
          .to_csv(dir_ + 'top_countries_work_count.csv', header=True, index=False))

In [100]:
# Utilized cached sparked dataframe for top countries citation count 

(df_school.where(F.col('country_code') != 'null')
          .groupby(['country_code']).agg(F.sum('cited_by_count')
          .alias('cited_by_count'))
          .sort(F.desc('cited_by_count')).toPandas())

,country_code,cited_by_count
0,US,195869
1,CN,97545
2,JP,50148
3,GB,44397
4,DE,35677
...,...,...
169,SO,0
170,GM,0
171,TM,0
172,AF,0


In [29]:
# Utilized cached sparked dataframe for top countries citation count export

(df_school.where(F.col('country_code') != 'null')
          .groupby(['country_code']).agg(F.sum('cited_by_count')
          .alias('cited_by_count'))
          .sort(F.desc('cited_by_count')).toPandas()
          .to_csv(dir_ + 'top_countries_cited_by_count.csv', header=True, index=False))

In [105]:
# Utilized cached sparked dataframe for top countries citation count > 100

(df_school.where((F.col('country_code') != 'null') & (F.col('cited_by_count') > 100))
          .groupby(['country_code']).agg(F.count('cited_by_count')
          .alias('cited_by_count'))
          .sort(F.desc('cited_by_count')).toPandas())

,country_code,cited_by_count
0,US,180
1,JP,47
2,DE,41
3,GB,29
4,CN,26
5,NL,14
6,ES,14
7,FR,14
8,CA,13
9,IT,12


In [30]:
# Utilized cached sparked dataframe for top countries citation count > 100 export

(df_school.where((F.col('country_code') != 'null') & (F.col('cited_by_count') > 100))
          .groupby(['country_code']).agg(F.count('cited_by_count')
          .alias('cited_by_count'))
          .sort(F.desc('cited_by_count')).toPandas()
          .to_csv(dir_ + 'top_countries_cited_by_count_100count.csv', header=True, index=False))

In [117]:
# Utilized cached sparked dataframe for top countries work count > 20

(df_school.where((F.col('country_code') != 'null') & (F.col('works_count') > 20))
          .groupby(['country_code']).agg(F.count('works_count')
          .alias('works_count'))
          .sort(F.desc('works_count')).toPandas())

,country_code,works_count
0,US,77
1,CN,44
2,JP,36
3,ES,26
4,RU,25
5,ID,23
6,FR,21
7,IT,20
8,BR,19
9,GB,18


In [31]:
# Utilized cached sparked dataframe for top countries work count > 20 export

(df_school.where((F.col('country_code') != 'null') & (F.col('works_count') > 20))
          .groupby(['country_code']).agg(F.count('works_count')
          .alias('works_count'))
          .sort(F.desc('works_count')).toPandas()
          .to_csv(dir_ + 'top_countries_work_by_count_20count.csv', header=True, index=False))

In [119]:
# Utilized cached sparked dataframe for top institution work count > 20

(df_school.where((F.col('works_count') > 20))
          .groupby(['id', 'display_name','country_code']).agg(F.count('works_count')
          .alias('works_count'))
          .sort(F.desc('works_count')).toPandas())

,id,display_name,country_code,works_count
0,https://openalex.org/I135598925,Kyushu University,JP,9
1,https://openalex.org/I1313323035,Russian Academy of Sciences,RU,9
2,https://openalex.org/I118501908,National Research University Higher School of ...,RU,9
3,https://openalex.org/I47720641,Huazhong University of Science and Technology,CN,4
4,https://openalex.org/I181343428,University of Zagreb,HR,4
...,...,...,...,...
397,https://openalex.org/I2613432,University of South Florida,US,1
398,https://openalex.org/I2960094004,Hospital General Universitario Gregorio Marañón,ES,1
399,https://openalex.org/I170230895,Warsaw University of Life Sciences,PL,1
400,https://openalex.org/I46597724,Poznań University of Technology,PL,1


In [32]:
# Utilized cached sparked dataframe for top institution work count > 20 export

(df_school.where((F.col('works_count') > 20))
          .groupby(['id', 'display_name','country_code']).agg(F.count('works_count')
          .alias('works_count'))
          .sort(F.desc('works_count')).toPandas()
          .to_csv(dir_ + 'top_insti_country_work_by_count_20count.csv', header=True, index=False))

In [123]:
# Utilized cached sparked dataframe for top institution citation count > 20

(df_school.where((F.col('cited_by_count') > 100))
          .groupby(['id', 'display_name','country_code']).agg(F.count('cited_by_count')
          .alias('cited_by_count'))
          .sort(F.desc('cited_by_count')).toPandas())

,id,display_name,country_code,cited_by_count
0,https://openalex.org/I19820366,Chinese Academy of Sciences,CN,5
1,https://openalex.org/I8204097,Ludwig-Maximilians-Universität München,DE,5
2,https://openalex.org/I114531698,Tokyo Institute of Technology,JP,5
3,https://openalex.org/I1294671590,French National Centre for Scientific Research,FR,5
4,https://openalex.org/I201537933,Tohoku University,JP,5
...,...,...,...,...
375,https://openalex.org/I47529993,Norwegian Institute of Technology,NO,1
376,https://openalex.org/I14243506,Hong Kong Polytechnic University,CN,1
377,https://openalex.org/I72702400,Athens State University,US,1
378,https://openalex.org/I5936373,University of Montenegro,ME,1


In [33]:
# Utilized cached sparked dataframe for top institution citation count > 20 export

(df_school.where((F.col('cited_by_count') > 100))
          .groupby(['id', 'display_name','country_code']).agg(F.count('cited_by_count')
          .alias('cited_by_count'))
          .sort(F.desc('cited_by_count')).toPandas()
          .to_csv(dir_ + 'top_insti_country_cited_by_count_100count.csv', header=True, index=False))

In [134]:
# Top 100 concepts based on authors count

(df2.withColumn('expl_x_concepts', F.explode('x_concepts'))
    .select(
             F.col('expl_x_concepts').getItem('display_name').alias('concept_display_name'), 
           )
    .groupby('concept_display_name').count()
    .sort(F.desc('count'))
    .limit(100)
    .toPandas()
    )



,concept_display_name,count
0,Biology,221052
1,Medicine,173714
2,Physics,167444
3,Computer science,165833
4,Chemistry,147839
...,...,...
95,Literature,11524
96,Anesthesia,11243
97,Theology,11172
98,Pedagogy,11143


In [34]:
# Top 100 concepts based on authors count export

(df2.withColumn('expl_x_concepts', F.explode('x_concepts'))
    .select(
             F.col('expl_x_concepts').getItem('display_name').alias('concept_display_name'), 
           )
    .groupby('concept_display_name').count()
    .sort(F.desc('count'))
    .limit(100)
    .toPandas()
    .to_csv(dir_ + 'top_100_topics.csv', header=True, index=False)
    )



In [144]:
# Top 100 concepts based on authors count that is level 4 and 5

(df2.withColumn('expl_x_concepts', F.explode('x_concepts'))
    .select(
             F.col('expl_x_concepts').getItem('display_name').alias('concept_display_name'), 
             F.col('expl_x_concepts').getItem('level').alias('level')
           )
    .where(F.col('level').isin([4,5]))
    .groupby('concept_display_name').count()
    .sort(F.desc('count'))
    .limit(20)
    .toPandas()
    )


,concept_display_name,count
0,Coronavirus disease 2019 (COVID-19),5297
1,Pandemic,2229
2,Severe acute respiratory syndrome coronavirus ...,2204
3,Interface (matter),950
4,Single-nucleotide polymorphism,699
5,Transcriptome,629
6,Repressor,485
7,Alpha (finance),427
8,Translation (biology),419
9,Operator (biology),414


In [4]:
# Top 100 concepts based on authors count that is level 4 and 5 export

(df2.withColumn('expl_x_concepts', F.explode('x_concepts'))
    .select(
             F.col('expl_x_concepts').getItem('display_name').alias('concept_display_name'), 
             F.col('expl_x_concepts').getItem('level').alias('level')
           )
    .where(F.col('level').isin([4,5]))
    .groupby('concept_display_name').count()
    .sort(F.desc('count'))
    .limit(20)
    .toPandas()
    .to_csv(dir_ + 'top_level4and5_20_topics.csv', header=True, index=False)
    )


In [150]:
# Top 20 instituion based on authors count that is level 4 and 5

(df2.withColumn('expl_x_concepts', F.explode('x_concepts'))
    .select( 'last_known_institution.id',
             'last_known_institution.display_name',
             F.col('expl_x_concepts').getItem('display_name').alias('concept_display_name'), 
             F.col('expl_x_concepts').getItem('level').alias('level')
           )
    .where(F.col('level').isin([4,5]) & (F.col('display_name') != 'null'))
    .groupby('id', 'display_name', 'concept_display_name').count()
    .sort(F.desc('count'))
    .limit(20)
    .toPandas()
    )


,id,display_name,concept_display_name,count
0,https://openalex.org/I31419693,Kyungpook National University,Muon collider,13
1,https://openalex.org/I1288882113,Boston Children's Hospital,Coronavirus disease 2019 (COVID-19),12
2,https://openalex.org/I51317819,Institute of Medical Sciences,Coronavirus disease 2019 (COVID-19),10
3,https://openalex.org/I19820366,Chinese Academy of Sciences,Transcriptome,8
4,https://openalex.org/I27837315,University of Michigan–Ann Arbor,Coronavirus disease 2019 (COVID-19),8
5,https://openalex.org/I98704320,Icahn School of Medicine at Mount Sinai,Coronavirus disease 2019 (COVID-19),8
6,https://openalex.org/I1288882113,Boston Children's Hospital,Pandemic,7
7,https://openalex.org/I89652312,Northwest A&F University,Transcriptome,7
8,https://openalex.org/I1289490764,Centers for Disease Control and Prevention,Coronavirus disease 2019 (COVID-19),7
9,https://openalex.org/I45129253,University College London,Coronavirus disease 2019 (COVID-19),6


In [5]:
# Top 20 instituion based on authors count that is level 4 and 5 export

(df2.withColumn('expl_x_concepts', F.explode('x_concepts'))
    .select( 'last_known_institution.id',
             'last_known_institution.display_name',
             F.col('expl_x_concepts').getItem('display_name').alias('concept_display_name'), 
             F.col('expl_x_concepts').getItem('level').alias('level')
           )
    .where(F.col('level').isin([4,5]) & (F.col('display_name') != 'null'))
    .groupby('id', 'display_name', 'concept_display_name').count()
    .sort(F.desc('count'))
    .limit(20)
    .toPandas()
    .to_csv(dir_ + 'top_schools_for_topics_level4and5.csv', header=True, index=False)
    )


In [ ]:
# 'last_known_institution.id',
# 'last_known_institution.display_name',
# 'last_known_institution.country_code',
# 'last_known_institution.ror',
# 'last_known_institution.type',
# 'works_count',
# 'cited_by_count'

In [153]:
# Top 10 concepts and institution of the authors 

(df2.withColumn('expl_x_concepts', F.explode('x_concepts'))
    .select( 'last_known_institution.id',
             'last_known_institution.display_name',
             F.col('expl_x_concepts').getItem('display_name').alias('concept_display_name'), 
             F.col('expl_x_concepts').getItem('level').alias('level')
           )
    .where((F.col('display_name') != 'null'))
    .groupby('id', 'display_name', 'concept_display_name').count()
    .sort(F.desc('count'))
    .limit(20)
    .toPandas()
    )


,id,display_name,concept_display_name,count
0,https://openalex.org/I19820366,Chinese Academy of Sciences,Physics,647
1,https://openalex.org/I19820366,Chinese Academy of Sciences,Biology,596
2,https://openalex.org/I19820366,Chinese Academy of Sciences,Chemistry,591
3,https://openalex.org/I1313323035,Russian Academy of Sciences,Physics,579
4,https://openalex.org/I1313323035,Russian Academy of Sciences,Chemistry,546
5,https://openalex.org/I1313323035,Russian Academy of Sciences,Biology,539
6,https://openalex.org/I19820366,Chinese Academy of Sciences,Engineering,535
7,https://openalex.org/I19820366,Chinese Academy of Sciences,Quantum mechanics,482
8,https://openalex.org/I1313323035,Russian Academy of Sciences,Materials science,439
9,https://openalex.org/I1313323035,Russian Academy of Sciences,Organic chemistry,438


In [6]:
# Top 10 concepts and institution of the authors export

(df2.withColumn('expl_x_concepts', F.explode('x_concepts'))
    .select( 'last_known_institution.id',
             'last_known_institution.display_name',
             F.col('expl_x_concepts').getItem('display_name').alias('concept_display_name'), 
             F.col('expl_x_concepts').getItem('level').alias('level')
           )
    .where((F.col('display_name') != 'null'))
    .groupby('id', 'display_name', 'concept_display_name').count()
    .sort(F.desc('count'))
    .limit(20)
    .toPandas()
    .to_csv(dir_ + 'top_school_concepts.csv', header=True, index=False)
    )


In [162]:
# Topics list related to pandemic or corona in the concept based on author count

(df2.withColumn('expl_x_concepts', F.explode('x_concepts'))
    .select( 
             F.col('expl_x_concepts').getItem('display_name').alias('concept_display_name'), 

           )
    .where((F.lower(F.col('concept_display_name')).like('%pandemic%') | 
            F.lower(F.col('concept_display_name')).like('%corona%')))
    .groupby('concept_display_name').count()
    .sort(F.desc('count'))
    .limit(20)
    .toPandas()
    )


,concept_display_name,count
0,Coronavirus disease 2019 (COVID-19),5297
1,Pandemic,2229
2,Severe acute respiratory syndrome coronavirus ...,2204
3,2019-20 coronavirus outbreak,1842
4,Coronary artery disease,669
5,Percutaneous coronary intervention,420
6,Coronavirus,383
7,Coronary heart disease,313
8,Acute coronary syndrome,247
9,Coronary angiography,239


In [7]:
# Topics list related to pandemic or corona in the concept based on author count export

(df2.withColumn('expl_x_concepts', F.explode('x_concepts'))
    .select( 
             F.col('expl_x_concepts').getItem('display_name').alias('concept_display_name'), 

           )
    .where((F.lower(F.col('concept_display_name')).like('%pandemic%') | 
            F.lower(F.col('concept_display_name')).like('%corona%')))
    .groupby('concept_display_name').count()
    .sort(F.desc('count'))
    .limit(20)
    .toPandas()
    .to_csv(dir_ + 'total_pandemic.csv', header=True, index=False)
    )


In [194]:
# Level distribution of authors count and percentage

df_per = df2.withColumn('expl_x_concepts', F.explode('x_concepts'))
                        
(df_per.select( 
             F.col('expl_x_concepts').getItem('level').alias('level')
           )
       .groupby('level').count()
       .select(F.col('count'), F.lit(F.col('count') / df_per.count() * 100).alias('distrbution'))
       .sort('level')
       .toPandas()
    )


,count,distrbution
0,2054058,27.378533
1,2986073,39.801358
2,1708272,22.769552
3,576759,7.687619
4,143321,1.910325
5,33957,0.452613


In [8]:
# Level distribution of authors count and percentage export

df_per = df2.withColumn('expl_x_concepts', F.explode('x_concepts'))
                        
(df_per.select( 
             F.col('expl_x_concepts').getItem('level').alias('level')
           )
       .groupby('level').count()
       .select(F.col('count'), F.lit(F.col('count') / df_per.count() * 100).alias('distrbution'))
       .sort('level')
       .toPandas()
       .to_csv(dir_ + 'level_distribution.csv', header=True, index=False)
    )


In [198]:
# Authors count related to data science machine learning, a.i, and artificial intelligence

(df2.withColumn('expl_x_concepts', F.explode('x_concepts'))
    .select( 
             F.col('expl_x_concepts').getItem('display_name').alias('concept_display_name'), 

           )
    .where((F.lower(F.col('concept_display_name')).like('%data science%') | 
            F.lower(F.col('concept_display_name')).like('%machine learning%') |
            F.lower(F.col('concept_display_name')).like('%a.i.%') |
            F.lower(F.col('concept_display_name')).like('%artificial intelligence%')
           ))
    .groupby('concept_display_name').count()
    .sort(F.desc('count'))
    .limit(20)
    .toPandas()
    
    )


,concept_display_name,count
0,Artificial intelligence,42167
1,Machine learning,21217
2,Data science,4121
3,Margin (machine learning),203
4,Boosting (machine learning),94
5,Active learning (machine learning),81
6,Anticipation (artificial intelligence),51
7,Semantic analysis (machine learning),10
8,Applications of artificial intelligence,6
9,Artificial Intelligence System,5


In [9]:
# Authors count related to data science machine learning, a.i, and artificial intelligence export

(df2.withColumn('expl_x_concepts', F.explode('x_concepts'))
    .select( 
             F.col('expl_x_concepts').getItem('display_name').alias('concept_display_name'), 

           )
    .where((F.lower(F.col('concept_display_name')).like('%data science%') | 
            F.lower(F.col('concept_display_name')).like('%machine learning%') |
            F.lower(F.col('concept_display_name')).like('%a.i.%') |
            F.lower(F.col('concept_display_name')).like('%artificial intelligence%')
           ))
    .groupby('concept_display_name').count()
    .sort(F.desc('count'))
    .toPandas()
    .to_csv(dir_ + 'data_science.csv', header=True, index=False)
    )


In [216]:
# % of Data Science Authors

(df2.withColumn('expl_x_concepts', F.explode('x_concepts'))
    .select( 
             F.col('expl_x_concepts').getItem('display_name').alias('concept_display_name'), 

           )
    .where((F.lower(F.col('concept_display_name')).like('%data science%') | 
            F.lower(F.col('concept_display_name')).like('%machine learning%') |
            F.lower(F.col('concept_display_name')).like('%a.i.%') |
            F.lower(F.col('concept_display_name')).like('%artificial intelligence%')
           ))
    .groupby('concept_display_name').count()
    .select(((F.sum(F.col('count')) / df_per.count()) *100).alias('Data Science %'))
    .toPandas()
    )


,Data Science %
0,0.905959


In [10]:
# % of Data Science Authors Export

(df2.withColumn('expl_x_concepts', F.explode('x_concepts'))
    .select( 
             F.col('expl_x_concepts').getItem('display_name').alias('concept_display_name'), 

           )
    .where((F.lower(F.col('concept_display_name')).like('%data science%') | 
            F.lower(F.col('concept_display_name')).like('%machine learning%') |
            F.lower(F.col('concept_display_name')).like('%a.i.%') |
            F.lower(F.col('concept_display_name')).like('%artificial intelligence%')
           ))
    .groupby('concept_display_name').count()
    .select(((F.sum(F.col('count')) / df_per.count()) *100).alias('Data Science %'))
    .toPandas()
    .to_csv(dir_ + 'data_science_percentage.csv', header=True, index=False)
    )


### Country Dictionary Export to pickle file

In [9]:
import pickle

# If None it does not exist on the geopandas map dataset
country_dict = {
    'AF': 'Afghanistan',
    'AO': None,
    'AE': 'United Arab Emirates',
    'AG': None,
    'AL': 'Albania',
    'AM': 'Armenia',
    'AO': 'Angola',
    'AR' : 'Argentina',
    'AT': 'Austria',
    'AU': 'Australia',
    'AW': None,
    'AX': None,
    'AZ': 'Azerbaijan',
    'BA': 'Bosnia and Herz.',
    'BB': None,
    'BD': 'Bangladesh',
    'BE': 'Belgium',
    'BF': 'Burkina Faso',
    'BG': 'Bulgaria',
    'BH': None,
    'BI': 'Burundi',
    'BJ': 'Benin',
    'BM': None,
    'BN': 'Brunei',
    'BO': 'Bolivia',
    'BQ': None,
    'BR': 'Brazil',
    'BS': 'Bahamas',
    'BT': 'Bhutan',
    'BW': 'Botswana',
    'BY': 'Belarus',
    'BZ': 'Belize',
    'CA': 'Canada',
    'CD': 'Dem. Rep. Congo',
    'CF': 'Central African Rep.',
    'CG': 'Congo',
    'CH': 'Switzerland',
    'CI': "Côte d'Ivoire",
    'CK': None,
    'CL': 'Chile',
    'CM': 'Cameroon',
    'CN': 'China',
    'CO': 'Colombia',
    'CR': 'Costa Rica',
    'CU': 'Cuba',
    'CV': None,
    'CW': None,
    'CY': 'Cyprus',
    'CZ': 'Czechia',
    'DE': 'Germany',
    'DJ': 'Djibouti',
    'DK': 'Denmark',
    'DM': None,
    'DO': 'Dominican Rep.',
    'DZ': 'Algeria',
    'EC': 'Ecuador',
    'EE': 'Estonia',
    'EG': 'Egypt',
    'ER': 'Eritrea',
    'ES': 'Spain',
    'ET': 'Ethiopia',
    'FI': 'Finland',
    'FJ': 'Fiji',
    'FK': 'Falkland Is.',
    'FM': None,
    'FO': None,
    'FR': 'France',
    'GA': 'Gabon',
    'GB': 'United Kingdom',
    'GD': None,
    'GE': 'Georgia',
    'GF': 'Guinea',
    'GH': 'Ghana',
    'GI': None,
    'GL': 'Greenland',
    'GM': 'Gambia',
    'GN': None,
    'GP': None,
    'GQ': 'Eq. Guinea',
    'GR': 'Greece',
    'GT': 'Guatemala',
    'GW': 'Guinea-Bissau',
    'GY': 'Guyana',
    'HK': None,
    'HN': 'Honduras',
    'HR': 'Croatia',
    'HT': 'Haiti',
    'HU': 'Hungary',
    'ID': 'Indonesia',
    'IE': 'Ireland',
    'IL': 'Israel',
    'IM': None,
    'IN': 'India',
    'IQ': 'Iraq',
    'IR': 'Iran',
    'IS': 'Iceland',
    'IT': 'Italy',
    'JE': None,
    'JM': 'Jamaica',
    'JO': 'Jordan',
    'JP': 'Japan',
    'KE': 'Kenya',
    'KG': 'Kyrgyzstan',
    'KH': 'Cambodia',
    'KI': None,
    'KN': None,
    'KP': 'North Korea',
    'KR': 'South Korea',
    'KW': 'Kuwait',
    'KY': None,
    'KZ': 'Kazakhstan',
    'LA': 'Laos',
    'LB': 'Lebanon',
    'LC': None,
    'LI': None,
    'LK': 'Sri Lanka',
    'LR': 'Liberia',
    'LS': 'Lesotho',
    'LT': 'Lithuania',
    'LU': 'Luxembourg',
    'LV': 'Latvia',
    'LY': 'Libya',
    'MA': 'Morocco',
    'MC': None,
    'MD': 'Moldova',
    'ME': 'Montenegro',
    'MG': 'Madagascar',
    'MH': None,
    'MK': 'North Macedonia',
    'ML': 'Mali',
    'MM': 'Myanmar',
    'MN': 'Mongolia',
    'MO': None,
    'MQ': None,
    'MR': 'Mauritania',
    'MS': None,
    'MT': None,
    'MU': None,
    'MV': None,
    'MW': 'Malawi',
    'MX': 'Mexico',
    'MY': 'Malaysia',
    'MZ': 'Mozambique',
    'NC': 'New Caledonia',
    'NE': 'Niger',
    'NG': 'Nigeria',
    'NI': 'Nicaragua',
    'NL': 'Netherlands',
    'NO': 'Norway',
    'NP': 'Nepal',
    'NU': None,
    'NZ': 'New Zealand',
    'OM': 'Oman',
    'PA': 'Panama',
    'PE': 'Peru',
    'PF': None,
    'PG': 'Papua New Guinea',
    'PH': 'Philippines',
    'PK': 'Pakistan',
    'PL': 'Poland',
    'PR': 'Puerto Rico',
    'PS': 'Palestine',
    'PT': 'Portugal',
    'PW': None,
    'PY': 'Paraguay',
    'QA': 'Qatar',
    'RE': None,
    'RO': 'Romania',
    'RS': 'Serbia',
    'RU': 'Russia',
    'RW': 'Rwanda',
    'SA': 'Saudi Arabia',
    'SB': 'Solomon Is.',
    'SC': None,
    'SD': 'Sudan',
    'SE': 'Sweden',
    'SG': None,
    'SI': 'Slovenia',
    'SJ': None,
    'SK': 'Slovakia',
    'SL': 'Sierra Leone',
    'SM': None,
    'SN': 'Senegal',
    'SO': 'Somalia',
    'SR': 'Suriname',
    'SS': 'S. Sudan',
    'ST': None,
    'SV': 'El Salvador',
    'SX': None,
    'SY': 'Syria',
    'SZ': 'eSwatini',
    'TC': None,
    'TD': 'Chad',
    'TG': 'Togo',
    'TH': 'Thailand',
    'TJ': 'Tajikistan',
    'TL': 'Timor-Leste',
    'TM': 'Turkmenistan',
    'TN': 'Tunisia',
    'TO': None,
    'TR': 'Turkey',
    'TT': 'Trinidad and Tobago',
    'TV': None,
    'TW': 'Taiwan',
    'TZ': 'Tanzania',
    'UA': 'Ukraine',
    'UG': 'Uganda',
    'US': 'United States of America',
    'UY': 'Uruguay',
    'UZ': 'Uzbekistan',
    'VA': None,
    'VC': None,
    'VE': 'Venezuela',
    'VG': None,
    'VI': None,
    'VN': 'Vietnam',
    'VU': 'Vanuatu',
    'WS': None,
    'XK': 'Kosovo',
    'YE': 'Yemen',
    'ZA': 'South Africa',
    'ZM': 'Zambia',
    'ZW': 'Zimbabwe'
    
}

# Used for the main dataset display
country_dict_df = {
    'AD': 'Andorra',
    'AE': 'United Arab Emirates',
    'AF': 'Afghanistan',
    'AG': 'Antigua and Barbuda',
    'AL': 'Albania',
    'AM': 'Armenia',
    'AO': 'Angola',
    'AR': 'Argentina',
    'AT': 'Austria',
    'AU': 'Australia',
    'AW': 'Aruba',
    'AX': 'Åland Islands',
    'AZ': 'Azerbaijan',
    'BA': 'Bosnia and Herzegovina',
    'BB': 'Barbados',
    'BD': 'Bangladesh',
    'BE': 'Belgium',
    'BF': 'Burkina Faso',
    'BG': 'Bulgaria',
    'BH': 'Bahrain',
    'BI': 'Burundi',
    'BJ': 'Benin',
    'BM': 'Bermuda',
    'BN': 'Brunei Darussalam',
    'BO': 'Bolivia',
    'BQ': 'Bonaire, Sint Eustatius and Saba',
    'BR': 'Brazil',
    'BS': 'Bahamas',
    'BT': 'Bhutan',
    'BW': 'Botswana',
    'BY': 'Belarus',
    'BZ': 'Belize',
    'CA': 'Canada',
    'CD': 'Democratic Republic of the Congo',
    'CF': 'Central African Republic',
    'CG': 'Republic of the Congo',
    'CH': 'Switzerland',
    'CI': "Côte d'Ivoire",
    'CK': 'Cook Islands',
    'CL': 'Chile',
    'CM': 'Cameroon',
    'CN': 'China',
    'CO': 'Colombia',
    'CR': 'Costa Rica',
    'CU': 'Cuba',
    'CV': 'Cape Verde',
    'CW': 'Curaçao',
    'CY': 'Cyprus',
    'CZ': 'Czech Republic',
    'DE': 'Germany',
    'DJ': 'Djibouti',
    'DK': 'Denmark',
    'DM': 'Dominica',
    'DO': 'Dominican Republic',
    'DZ': 'Algeria',
    'EC': 'Ecuador',
    'EE': 'Estonia',
    'EG': 'Egypt',
    'ER': 'Eritrea',
    'ES': 'Spain',
    'ET': 'Ethiopia',
    'FI': 'Finland',
    'FJ': 'Fiji',
    'FK': 'Falkland Islands',
    'FM': 'Federated States of Micronesia',
    'FO': 'Faroe Islands',
    'FR': 'France',
    'GA': 'Gabon',
    'GB': 'United Kingdom',
    'GD': 'Grenada',
    'GE': 'Georgia',
    'GF': 'French Guiana',
    'GH': 'Ghana',
    'GI': 'Gibraltar',
    'GL': 'Greenland',
    'GM': 'Gambia',
    'GN': 'Guinea',
    'GP': 'Guadeloupe',
    'GQ': 'Equatorial Guinea',
    'GR': 'Greece',
    'GT': 'Guatemala',
    'GW': 'Guinea-Bissau',
    'GY': 'Guyana',
    'HK': 'Hong Kong',
    'HN': 'Honduras',
    'HR': 'Croatia',
    'HT': 'Haiti',
    'HU': 'Hungary',
    'ID': 'Indonesia',
    'IE': 'Ireland',
    'IL': 'Israel',
    'IM': 'Isle of Man',
    'IN': 'India',
    'IQ': 'Iraq',
    'IR': 'Iran',
    'IS': 'Iceland',
    'IT': 'Italy',
    'JE': 'Jersey',
    'JM': 'Jamaica',
    'JO': 'Jordan',
    'JP': 'Japan',
    'KE': 'Kenya',
    'KG': 'Kyrgyzstan',
    'KH': 'Cambodia',
    'KI': 'Kiribati',
    'KN': 'Saint Kitts and Nevis',
    'KP': 'North Korea',
    'KR': 'South Korea',
    'KW': 'Kuwait',
    'KY': 'Cayman Islands',
    'KZ': 'Kazakhstan',
    'LA': 'Laos',
    'LB': 'Lebanon',
    'LC': 'Saint Lucia',
    'LI': 'Liechtenstein',
    'LK': 'Sri Lanka',
    'LR': 'Liberia',
    'LS': 'Lesotho',
    'LT': 'Lithuania',
    'LU': 'Luxembourg',
    'LV': 'Latvia',
    'LY': 'Libya',
    'MA': 'Morocco',
    'MC': 'Monaco',
    'MD': 'Moldova',
    'ME': 'Montenegro',
    'MG': 'Madagascar',
    'MH': 'Marshall Islands',
    'MK': 'North Macedonia',
    'ML': 'Mali',
    'MM': 'Myanmar (Burma)',
    'MN': 'Mongolia',
    'MO': 'Macao',
    'MQ': 'Martinique',
    'MR': 'Mauritania',
    'MS': 'Montserrat',
    'MT': 'Malta',
    'MU': 'Mauritius',
    'MV': 'Maldives',
    'MW': 'Malawi',
    'MX': 'Mexico',
    'MY': 'Malaysia',
    'MZ': 'Mozambique',
    'NC': 'New Caledonia',
    'NE': 'Niger',
    'NG': 'Nigeria',
    'NI': 'Nicaragua',
    'NL': 'Netherlands',
    'NO': 'Norway',
    'NP': 'Nepal',
    'NU': 'Niue',
    'NZ': 'New Zealand',
    'OM': 'Oman',
    'PA': 'Panama',
    'PE': 'Peru',
    'PF': 'French Polynesia',
    'PG': 'Papua New Guinea',
    'PH': 'Philippines',
    'PK': 'Pakistan',
    'PL': 'Poland',
    'PR': 'Puerto Rico',
    'PS': 'Palestine',
    'PT': 'Portugal',
    'PW': 'Palau',
    'PY': 'Paraguay',
    'QA': 'Qatar',
    'RE': 'Réunion',
    'RO': 'Romania',
    'RS': 'Serbia',
    'RU': 'Russia',
    'RW': 'Rwanda',
    'SA': 'Saudi Arabia',
    'SB': 'Solomon Islands',
    'SC': 'Seychelles',
    'SD': 'Sudan',
    'SE': 'Sweden',
    'SG': 'Singapore',
    'SI': 'Slovenia',
    'SJ': 'Svalbard and Jan Mayen',
    'SK': 'Slovakia',
    'SL': 'Sierra Leone',
    'SM': 'San Marino',
    'SN': 'Senegal',
    'SO': 'Somalia',
    'SR': 'Suriname',
    'SS': 'South Sudan',
    'ST': 'Sao Tome and Principe',
    'SV': 'El Salvador',
    'SX': 'Sint Maarten (Dutch part)',
    'SY': 'Syria',
    'SZ': 'Eswatini',
    'TC': 'Turks and Caicos Islands',
    'TD': 'Chad',
    'TG': 'Togo',
    'TH': 'Thailand',
    'TJ': 'Tajikistan',
    'TL': 'Timor-Leste',
    'TM': 'Turkmenistan',
    'TN': 'Tunisia',
    'TO': 'Tonga',
    'TR': 'Turkey',
    'TT': 'Trinidad and Tobago',
    'TV': 'Tuvalu',
    'TW': 'Taiwan',
    'TZ': 'Tanzania',
    'UA': 'Ukraine',
    'UG': 'Uganda',
    'US': 'United States',
    'UY': 'Uruguay',
    'UZ': 'Uzbekistan',
    'VA': 'Vatican City',
    'VC': 'Saint Vincent and the Grenadines',
    'VE': 'Venezuela',
    'VG': 'British Virgin Islands',
    'VI': 'U.S. Virgin Islands',
    'VN': 'Vietnam',
    'VU': 'Vanuatu',
    'WS': 'Samoa',
    'XK': 'Kosovo',
    'YE': 'Yemen',
    'ZA': 'South Africa',
    'ZM': 'Zambia',
    'ZW': 'Zimbabwe'
}

# export to pickle
with open('country_dict.pkl', 'wb') as file:
    pickle.dump(country_dict, file)

with open('country_dict_df.pkl', 'wb') as file:
    pickle.dump(country_dict, file)